## MATLAB Simulated Arm Control
While ROS has many great packages for robot visualiztion and simulation, we cant use them since we are running ROS in a jupyter notebook which does not have a desktop gui. To get around this and still give everyone a chance to iteract with a simulated robot using ROS, I developed a MATLAB program whcih launches a virtaul UR3 arm which can be controlled via ROS. It can be run in desktop MATLAB or MATLAB online. In the MATLAB program you will need to change the MQTT client name and pub/sub topics based on your group number.

### How it works
I created a ROS2 package which launches N versions of a node configured as an action server. It uses parameters to dynamically set MQTT client names, topics, and FollowJointTrajectory namespaces so each node can be linked to a single simulated arm. MQTT is used to connect to the simulation which is outside the ROS network (this is a little bit of a weird roundabout setup but not unheard of to use microcontrollers in ROS systems since they cant run the full thing). Also it means the same FollowJointTrajectory action client which is used to control the arm can be used to control the simulation just by changing the namespace of the action.

To get the MATLAB arm simulator function or see the inner workings of the package you can check out the [github repo](https://github.com/e-sarvey/ROS2-MATLAB-Simulator).


In [1]:
! ros2 node list # check the nodes availible

/simulated_arm_server_1
/simulated_arm_server_2


In [ ]:
! ros2 action list # check the actions availible

### Sending an action goal to the simulated arm
To send a goal to the simulated arm, edit the CLI command below for your action server (eg. /simulated_arm_server_N/). You can also change the positions to alter the joint angles. Play around with the angles and see if you can get the arm to move to a desired position just by setting the angles. (Hint: There is a commented bit of code in the matlab function that can print the end effector end positon for a given end effector offset (since the sim doesnt have an end effector on it). Ideally this would be set up as a service in the simulated arm node but I didnt have time to figure that out).

In [ ]:
! ros2 action send_goal /simulated_arm_server_1/follow_joint_trajectory control_msgs/action/FollowJointTrajectory "{trajectory: {joint_names: ['shoulder_pan_joint', 'shoulder_lift_joint', 'elbow_joint', 'wrist_1_joint', 'wrist_2_joint', 'wrist_3_joint'], \
points: [{positions: [0.5, -0.5, 0.3, 0.1, -0.1, 0.0]}]}}"

### Sending an action goal to the real arm
This part isnt finished yet :( I was hoping to be able to control the arm using the same  action client but I didnt figure out the additional configuration parameters yet. Then you can plan out a trajectory in the simulation and test it on the real arm by changing just a few values which highlights the modularity of ROS!

In [ ]:
! ros2 action send_goal /____/follow_joint_trajectory control_msgs/action/FollowJointTrajectory "{trajectory: {joint_names: ['shoulder_pan_joint', 'shoulder_lift_joint', 'elbow_joint', 'wrist_1_joint', 'wrist_2_joint', 'wrist_3_joint'], \
points: [{positions: [0.5, -0.5, 0.3, 0.1, -0.1, 0.0]}]}}"